In [1]:
%matplotlib widget

import os  
import sys  
sys.path.insert(0, os.getenv('HOME')+'/pycode/MscThesis/')
import pandas as pd
from amftrack.util.sys import get_dates_datetime, get_dirname, get_plate_number, get_postion_number

import ast
from amftrack.plotutil import plot_t_tp1
from scipy import sparse
from datetime import datetime
from amftrack.pipeline.functions.node_id import orient
import pickle
import scipy.io as sio
from pymatreader import read_mat
from matplotlib import colors
import cv2
import imageio
import matplotlib.pyplot as plt
import numpy as np
from skimage.filters import frangi
from skimage import filters
from random import choice
import scipy.sparse
import os
from amftrack.pipeline.functions.extract_graph import from_sparse_to_graph, generate_nx_graph, sparse_to_doc
from skimage.feature import hessian_matrix_det
from amftrack.pipeline.functions.experiment_class_surf import Experiment
from amftrack.pipeline.paths.directory import run_parallel, find_state, directory_scratch, directory_project, path_code

from amftrack.notebooks.analysis.data_info import *
import matplotlib.patches as mpatches
from statsmodels.stats import weightstats as stests
window=800
results={}
for treatment in treatments.keys():
    insts = treatments[treatment]
    for inst in insts:
        results[inst] = pickle.load(open(f'{path_code}/MscThesis/Results/straight_{window}_{inst}.pick', "rb"))
column_names = ["plate","inst", "treatment", "angle", "curvature","density","growth","speed","straightness","t","hyph","total_network"]
infos = pd.DataFrame(columns=column_names)
for treatment in treatments.keys():
    insts = treatments[treatment]
    for inst in insts:
        angles, curvatures, densities,growths,speeds,tortuosities,ts,hyphs,total_lengths = results[inst]
        for i,angle in enumerate(angles):
            new_line = pd.DataFrame(
                {   "plate": [plate_number[inst]],
                    "inst": [inst],
                    "treatment": [treatment],
                    "angle": [angle],
                    "curvature": [curvatures[i]],
                    "density": [densities[i]],
                    "growth": [growths[i]],
                    "speed": [speeds[i]],
                    "straightness": [tortuosities[i]],
                     "t": [ts[i]],
                     "hyph": [hyphs[i]],
                     "total_network" : [total_lengths[i]],
                }
            )  # index 0 for
            # mothers need to be modified to resolve multi mother issue
            infos = infos.append(new_line, ignore_index=True)
corrected = infos.loc[(infos["straightness"] <= 1)&(infos["plate"] !=435)& (infos["speed"] >=25)& (infos["speed"] <400)]
plate_94 = corrected.loc[corrected ['plate']==94]


In [2]:
plate_94 = corrected.loc[(corrected ['plate']==94) & (corrected ['t']<=70)& (corrected ['speed']>=50)]

# plate_94 = corrected.loc[corrected ['plate']==94]


In [2]:
plate_94 = corrected.loc[(corrected ['plate']==94) & (corrected ['t']<=70)]
# np.max(plate_94['total_network'])/1000

In [3]:
baits_sort = plate_94.sort_values("t")
N=20
moving_av = baits_sort.rolling(N).mean()
moving_std = baits_sort.rolling(N).std()
plt.rcParams.update({
    "font.family": "verdana",
'font.weight' : 'normal',
'font.size': 20})
fig=plt.figure(figsize=(8,8))
ax = fig.add_subplot(111)
ax.set_xlabel('time(t)')
ax.set_ylabel('speed of hyphal tips ($\mu m. h^{-1}$)')
xlim = 40
# ax.set_xlim(0,30)
ax.set_ylim(0,400)
# slope, intercept, r_value, p_value, std_err = stats.linregress(densities_sort,np.abs(curvatures_sort))
x= range(-xlim,xlim)
ts = list(set(plate_94["t"].values))
ts.sort()
averages =  [np.mean(plate_94.loc[plate_94["t"]==t]['speed']) for t in ts]
sts = [np.std(plate_94.loc[plate_94["t"]==t]['speed'])/np.sqrt(len(plate_94.loc[plate_94["t"]==t]['speed'])) for t in ts]
ax.scatter(plate_94["t"],plate_94['speed'],color='yellow',label='observed speeds')
plt.errorbar(ts,averages,yerr=sts, linestyle="None",capthick = 1)
ax.plot(ts,averages)
# ax.plot(moving_av["t"],moving_av['speed'],color='green',label = 'moving average')
# ax.plot(moving_av["t"],(moving_av['speed']+moving_std['speed']/np.sqrt(N)),color='red',label = 'moving average')
# ax.plot(moving_av["t"],(moving_av['speed']-moving_std['speed']/np.sqrt(N)),color='red',label = 'moving average')

/home/cbisot/anaconda3/envs/test/lib/python3.7/site-packages/matplotlib/__init__.py:880: MatplotlibDeprecationWarning: 
nbagg.transparent
  version, key, obj_type="rcparam", alternative=alt_key)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [44]:
(1.10+1.79)*1500/np.log(2)/24

260.58679176056904

In [3]:
baits_sort = plate_94.sort_values("total_network")
N=80
moving_av = baits_sort.rolling(N).mean()
moving_std = baits_sort.rolling(N).std()

fig=plt.figure(figsize=(8,8))
ax = fig.add_subplot(111)
ax.set_xlabel('total network($\mu m$)')
ax.set_ylabel('speed($\mu m. h^{-1}$)')
xlim = 40
# ax.set_xlim(0,60)
ax.set_ylim(0,400)
# slope, intercept, r_value, p_value, std_err = stats.linregress(densities_sort,np.abs(curvatures_sort))
x= range(-xlim,xlim)

ax.scatter(plate_94["total_network"],plate_94['speed'],color='yellow',label='observed speeds')
ax.plot(moving_av["total_network"],moving_av['speed'],color='green',label = 'moving average')
ax.plot(moving_av["total_network"],(moving_av['speed']+moving_std['speed']/np.sqrt(N)),color='red',label = 'moving average')
ax.plot(moving_av["total_network"],(moving_av['speed']-moving_std['speed']/np.sqrt(N)),color='red',label = 'moving average')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [12]:
160**2*np.log(2)**2/(2*1000**2)*24**2,56/20*np.log(2)/1000*24**2

(3.542283981016115, 1.1179077728070796)

In [11]:
160**2*np.log(2)**2/(2*1000**2)*24**2 + 3*np.log(2)/1000*24**2

4.7400423090237

In [39]:
np.sqrt(4*np.log(2)/1500*24**2)

1.0318304460230256

In [40]:
30000/3600

8.333333333333334

In [5]:
0.80*24

19.200000000000003